# L'Objectif du Projet

le but principal de ce projet est de générer un algorithme qui permet de proposer des recommandations de films aux utilisateurs en se basant sur la similarité entre les films ainsi que l'évaluation de ces films donné par les utilisateurs.

# Introduction:

Le Collaboratif Filtering:  se base sur la note attribué par l'utilisateur sur l'item puisque Il existe deux type de recommanders CF:
    -Memory Based:
    -Model Based:
Pour le modèle Memory Based, on a utilisé le modèle user based et item based pour les métriques "cosine similarity" et "cityblock"
"""""

## La similarité entre les films

nous commencerons par charger l'ensemble de données MovieLens En utilisant Pandas, et nous chargerons l'ensemble de données des évaluations et l'ensemble de données de films puis nous les fusionnerons ensemble pour que nous puissions travailler avec des noms de films au lieu de ID's

NB : vous pouvez télécharger les datasets à travers le site :https://movielens.org

# chargement des données

In [1]:
import pandas as pd
import numpy as np
# charger les datasets de films et des évaltaions

rat_cols= ['user_id','movie_id','rating']
ratings=pd.read_csv('C:/Users/Chhimy/Desktop/u.data', sep='\t', names=rat_cols, usecols=range(3), encoding='latin-1')

movie_cols= ['movie_id','title']
movies=pd.read_csv('C:/Users/Chhimy/Desktop/u.item', sep='|', names=movie_cols, usecols=range(2),encoding='latin-1')

#  fusionner les deux dataframes 

ratings=pd.merge(ratings,movies)
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [2]:
ratings.head()

,user_id,movie_id,rating,title
0,196,242,3,Kolya (1996)
1,63,242,3,Kolya (1996)
2,226,242,5,Kolya (1996)
3,154,242,3,Kolya (1996)
4,306,242,5,Kolya (1996)


Nous allons utiliser la fonction pivot_table() pour créer une matrice qui va prendre les utilisateurs en lignes et le titres des films en colonnes et comme valeurs les évaluations des utilisateurs pour chaque film 

NB: NaN indique les valeurs manquantes dans les datasets

In [3]:
movieRatings=ratings.pivot_table(index=['user_id'],columns=['title'],values=['rating'])
movieRatings=pd.DataFrame(movieRatings)
movieRatings.head()

rating                                     \
title   'Til There Was You (1997) 1-900 (1994) 101 Dalmatians (1996)   
user_id                                                                
1                             NaN          NaN                   2.0   
2                             NaN          NaN                   NaN   
3                             NaN          NaN                   NaN   
4                             NaN          NaN                   NaN   
5                             NaN          NaN                   2.0   

                                                                    \
title   12 Angry Men (1957) 187 (1997) 2 Days in the Valley (1996)   
user_id                                                              
1                       5.0        NaN                         NaN   
2                       NaN        NaN                         NaN   
3                       NaN        2.0                         NaN   
4                       NaN        NaN                         NaN   
5                       NaN        NaN                         NaN   

                                                                          \
title   20,000 Leagues Under the Sea (1954) 2001: A Space Odyssey (1968)   
user_id                                                                    
1                                       3.0                          4.0   
2                                       NaN                          NaN   
3                                       NaN                          NaN   
4                                       NaN                          NaN   
5                                       NaN                          4.0   

                                                                          \
title   3 Ninjas: High Noon At Mega Mountain (1998) 39 Steps, The (1935)   
user_id                                                                    
1                                               NaN                  NaN   
2                                               1.0                  NaN   
3                                               NaN                  NaN   
4                                               NaN                  NaN   
5                                               NaN                  NaN   

                       ...                                     \
title                  ...                 Yankee Zulu (1994)   
user_id                ...                                      
1                      ...                                NaN   
2                      ...                                NaN   
3                      ...                                NaN   
4                      ...                                NaN   
5                      ...                                NaN   

                                                      \
title   Year of the Horse (1997) You So Crazy (1994)   
user_id                                                
1                            NaN                 NaN   
2                            NaN                 NaN   
3                            NaN                 NaN   
4                            NaN                 NaN   
5                            NaN                 NaN   

                                                                          \
title   Young Frankenstein (1974) Young Guns (1988) Young Guns II (1990)   
user_id                                                                    
1                             5.0               3.0                  NaN   
2                             NaN               NaN                  NaN   
3                             NaN               NaN                  NaN   
4                             NaN               NaN                  NaN   
5                             4.0               NaN                  NaN   

                                                                               \
title   You

Exemple: nous allons extraire une liste des utilisateurs qui ont évalués le film 'Forrest Gump (1994) 

In [4]:
forrestgumpRatings = movieRatings['rating']['Forrest Gump (1994)']
# les 10 premiers résultats de l'évaluation
forrestgumpRatings.head(10)

                       


user_id
1     3.0
2     NaN
3     NaN
4     NaN
5     1.0
6     3.0
7     5.0
8     NaN
9     NaN
10    4.0
Name: Forrest Gump (1994), dtype: float64

Par la suite nous allons calculer la corrélation entre ce film et tous les films à travers la fonction corrwith pour mésurer la similarité entre eux 

In [5]:
similarMovies=movieRatings.corrwith(forrestgumpRatings)
# l'élimination des valeurs non numériques
similarMovies=similarMovies.dropna()
df=pd.DataFrame(similarMovies)


C:\Users\Chhimy\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\Chhimy\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


In [6]:
#l'affichage de valeur de corrélation entre Masterminds et 10 films. 
df.head(10)

0
       title                                        
rating 'Til There Was You (1997)           -0.132453
       1-900 (1994)                        -1.000000
       101 Dalmatians (1996)                0.215094
       12 Angry Men (1957)                  0.030303
       187 (1997)                           0.557086
       2 Days in the Valley (1996)          0.071940
       20,000 Leagues Under the Sea (1954)  0.243825
       2001: A Space Odyssey (1968)         0.077572
       39 Steps, The (1935)                 0.132406
       8 1/2 (1963)                        -0.184925

Le résultat semble illogique c'est grâce aux utilisateurs qui n'ont pas une grande nombre d'évaluations pour visualiser le problème nous allons créer un nouveau dataframe movieStats dans lequel on va stocker les films et le nombre des utilisateurs qui ont évalués le film ainsi qu'un moyen d'évaluation pour chaque film
dans le Machine Learning on appelle ce phénomène effet des valeurs aberrantes (outliers)

In [7]:
movieStats=ratings.groupby('title').agg({'rating':[np.size, np.mean]})
movieStats.head()
               

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

par la suite nous allons afficher les films qui ont plus de 50 évaluations 


In [8]:
popularMovies=movieStats['rating']['size']>=50
movieStats[popularMovies].sort_values([('rating','mean')],ascending=False)[:15]
#les 15 premiers résultat

rating          
                                                     size      mean
title                                                              
Close Shave, A (1995)                                 112  4.491071
Schindler's List (1993)                               298  4.466443
Wrong Trousers, The (1993)                            118  4.466102
Casablanca (1942)                                     243  4.456790
Wallace & Gromit: The Best of Aardman Animation...     67  4.447761
Shawshank Redemption, The (1994)                      283  4.445230
Rear Window (1954)                                    209  4.387560
Usual Suspects, The (1995)                            267  4.385768
Star Wars (1977)                                      583  4.358491
12 Angry Men (1957)                                   125  4.344000
Third Man, The (1949)                                  72  4.333333
Citizen Kane (1941)                                   198  4.292929
To Kill a Mockingbird (1962)                          219  4.292237
One Flew Over the Cuckoo's Nest (1975)                264  4.291667
Silence of the Lambs, The (1991)                      390  4.289744

112 ont évalués le film Close Shave, A (1995) d'un moyen d'évaluation le plus grand (4.49) 

nous allons combiner les résultat dans un seul dataframe df pour voir la similarité entre les films les plus évalués(plus de 50 évaluations) et le film Forrest Gump (1994)

In [9]:
df=movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))

C:\Users\Chhimy\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [10]:
df.sort_values(['similarity'], ascending=False)[:15]


(rating, size)  (rating, mean)  \
       title                                                                  
rating Forrest Gump (1994)                              321        3.853583   
       MatchMaker, The (1997)                            51        3.176471   
       Field of Dreams (1989)                           212        3.698113   
       Deconstructing Harry (1997)                       65        3.400000   
       Firm, The (1993)                                 151        3.278146   
       Weekend at Bernie's (1989)                        60        2.616667   
       Robin Hood: Prince of Thieves (1991)              75        3.226667   
       River Wild, The (1994)                           146        3.143836   
       Around the World in 80 Days (1956)                59        3.661017   
       Young Guns (1988)                                101        3.207921   
       Michael (1996)                                   119        3.319328   
       Barcelona (1994)                                  53        3.018868   
       Basic Instinct (1992)                            101        3.227723   
       Cool Runnings (1993)                              68        3.161765   
       Mimic (1997)                                     101        2.742574   

                                             similarity  
       title                                             
rating Forrest Gump (1994)                     1.000000  
       MatchMaker, The (1997)                  0.487950  
       Field of Dreams (1989)                  0.486450  
       Deconstructing Harry (1997)             0.474695  
       Firm, The (1993)                        0.464948  
       Weekend at Bernie's (1989)              0.456412  
       Robin Hood: Prince of Thieves (1991)    0.449048  
       River Wild, The (1994)                  0.448505  
       Around the World in 80 Days (1956)      0.443286  
       Young Guns (1988)                       0.437801  
       Michael (1996)                          0.426752  
       Barcelona (1994)                        0.423408  
       Basic Instinct (1992)                   0.420827  
       Cool Runnings (1993)                    0.413525  
       Mimic (1997)                            0.412722

le film Forrest Gump (1994) est corrélé à lui-même c normale mais pour les autres films on voie qu'il une corrélation acceptable 

# Item-based Filtering Collaborative

nous allons utilisé les mêmes données

# la corrélation entre les films

In [11]:
corrMatrix = movieRatings.corr(method='pearson', min_periods=50)


In [12]:
corrMatrix.head()

rating               \
title                            'Til There Was You (1997) 1-900 (1994)   
       title                                                              
rating 'Til There Was You (1997)                       NaN          NaN   
       1-900 (1994)                                    NaN          NaN   
       101 Dalmatians (1996)                           NaN          NaN   
       12 Angry Men (1957)                             NaN          NaN   
       187 (1997)                                      NaN          NaN   

                                                                            \
title                            101 Dalmatians (1996) 12 Angry Men (1957)   
       title                                                                 
rating 'Til There Was You (1997)                   NaN                 NaN   
       1-900 (1994)                                NaN                 NaN   
       101 Dalmatians (1996)                       1.0                 NaN   
       12 Angry Men (1957)                         NaN                 1.0   
       187 (1997)                                  NaN                 NaN   

                                                                         \
title                            187 (1997) 2 Days in the Valley (1996)   
       title                                                              
rating 'Til There Was You (1997)        NaN                         NaN   
       1-900 (1994)                     NaN                         NaN   
       101 Dalmatians (1996)            NaN                         NaN   
       12 Angry Men (1957)              NaN                         NaN   
       187 (1997)                       NaN                         NaN   

                                                                      \
title                            20,000 Leagues Under the Sea (1954)   
       title                                                           
rating 'Til There Was You (1997)                                 NaN   
       1-900 (1994)                                              NaN   
       101 Dalmatians (1996)                                     NaN   
       12 Angry Men (1957)                                       NaN   
       187 (1997)                                                NaN   

                                                               \
title                            2001: A Space Odyssey (1968)   
       title                                                    
rating 'Til There Was You (1997)                          NaN   
       1-900 (1994)                                       NaN   
       101 Dalmatians (1996)                              NaN   
       12 Angry Men (1957)                           0.178848   
       187 (1997)                                         NaN   

                                                                              \
title                            3 Ninjas: High Noon At Mega Mountain (1998)   
       title                                                                   
rating 'Til There Was You (1997)                                         NaN   
       1-900 (1994)                                                      NaN   
       101 Dalmatians (1996)                                             NaN   
       12 Angry Men (1957)                                               NaN   
       187 (1997)                                                        NaN   

                                                       \
title                            39 Steps, The (1935)   
       title                                            
rating 'Til There Was You (1997)                  NaN   
       1-900 (1994)                               NaN   
       101 Dalmatians (1996)                      NaN   
       12 Angry Men (1957)                        NaN   
       187 (1997)                                 NaN   

                             

pour faire la recommandations des films nous allons examiner toutes les évaluations donnés par un utilisateur et trouver les films similaires aux celles qui a déja évalué, ces films sont les meilleurs à recommander à cette personne

on prend un utilisateur par exemple de l'ID 5 et on tente de les récommander des films


In [13]:
rating1=movieRatings.loc[4].dropna()
rating1.head()
# il y a pas mal des films préférés par l'user 5 (Air Force One (1997) ,Assignment, The (1997), Blues Brothers 2000 (1998))

        title                     
rating  Air Force One (1997)          5.0
        Assignment, The (1997)        5.0
        Blues Brothers 2000 (1998)    5.0
        Client, The (1994)            3.0
        Conspiracy Theory (1997)      3.0
Name: 4, dtype: float64

on va créer une liste(similarCadidates) qui contient les films similaires o celles évalués par l'user de l'ID 5

In [14]:
similarCandidates = pd.Series()
for i in range(0, len(rating1.index)):
    print ("adding sims for"+ str(rating1.index[i]) + "...")
    sims=corrMatrix[rating1.index[i]].dropna()
    sims= sims.map(lambda x: x * rating1[i])
    similarCandidates=similarCandidates.append(sims)
print("Terminé")    


adding sims for('rating', 'Air Force One (1997)')...
adding sims for('rating', 'Assignment, The (1997)')...
adding sims for('rating', 'Blues Brothers 2000 (1998)')...
adding sims for('rating', 'Client, The (1994)')...
adding sims for('rating', 'Conspiracy Theory (1997)')...
adding sims for('rating', 'Contact (1997)')...
adding sims for('rating', 'Cop Land (1997)')...
adding sims for('rating', 'Desperate Measures (1998)')...
adding sims for('rating', 'Event Horizon (1997)')...
adding sims for('rating', 'In & Out (1997)')...
adding sims for('rating', 'Incognito (1997)')...
adding sims for('rating', 'Indiana Jones and the Last Crusade (1989)')...
adding sims for('rating', 'Liar Liar (1997)')...
adding sims for('rating', 'Lost Highway (1997)')...
adding sims for('rating', 'Mimic (1997)')...
adding sims for('rating', "One Flew Over the Cuckoo's Nest (1975)")...
adding sims for('rating', 'Scream (1996)')...
adding sims for('rating', 'Seven (Se7en) (1995)')...
adding sims for('rating', 'Spawn

In [15]:
#ordonner les résultat  par les meilleurs films
similarCandidates.sort_values(inplace=True, ascending=False)
print (similarCandidates.head(20))

(rating, Wedding Singer, The (1998))                5.000000
(rating, Contact (1997))                            5.000000
(rating, Star Wars (1977))                          5.000000
(rating, Air Force One (1997))                      5.000000
(rating, Ulee's Gold (1997))                        5.000000
(rating, Lost Highway (1997))                       5.000000
(rating, Liar Liar (1997))                          5.000000
(rating, In & Out (1997))                           5.000000
(rating, Desperate Measures (1998))                 5.000000
(rating, Cop Land (1997))                           5.000000
(rating, Seven (Se7en) (1995))                      4.000000
(rating, Scream (1996))                             4.000000
(rating, One Flew Over the Cuckoo's Nest (1975))    4.000000
(rating, Starship Troopers (1997))                  4.000000
(rating, Event Horizon (1997))                      4.000000
(rating, Empire Strikes Back, The (1980))           3.739907
(rating, Return of the J

Nous avons ordonnés la liste des films similaires aux ceux que l'utilisateur d'ID 5 préfère d'une manière décroissante.

In [16]:
similarCandidates.shape

(3994,)

donc nous allons recommander 550 films à l'utilisateur d'ID 5